In [1]:
import os
import sys

cwd = os.getcwd().split('/')
sys.path.append(os.path.join('/', *cwd[:cwd.index('discretizer')+1]))

In [2]:
from discretizer import momentum_operators
from discretizer.algorithms import magic

import sympy
import sympy.physics
from sympy.physics.quantum import TensorProduct as kr

In [3]:
sympy.init_printing(use_latex='mathjax')

In [4]:
s0 = sympy.eye(2)
sx = sympy.physics.matrices.msigma(1)
sy = sympy.physics.matrices.msigma(2)
sz = sympy.physics.matrices.msigma(3)

kx, ky, kz = momentum_operators
A, B, D, M, C = sympy.symbols('A B D M C', real=True)


In [5]:
k = sympy.sqrt(kx**2+ky**2)
hamiltonian = (+ C * kr(s0, s0) + M * kr(s0, sz) - D * k**2 * kr(s0, s0) 
               - B * k**2 * kr(s0, sz) + A * (kx * kr(sz, sx) - ky * kr(s0, sy)))

hamiltonian

⎡    ⎛  2      2⎞         ⎛  2      2⎞                                        
⎢- B⋅⎝kₓ  + k_y ⎠ + C - D⋅⎝kₓ  + k_y ⎠ + M              A⋅(kₓ + ⅈ⋅k_y)        
⎢                                                                             
⎢                                             ⎛  2      2⎞         ⎛  2      2
⎢             A⋅(kₓ - ⅈ⋅k_y)                B⋅⎝kₓ  + k_y ⎠ + C - D⋅⎝kₓ  + k_y 
⎢                                                                             
⎢                                                                             
⎢                    0                                         0              
⎢                                                                             
⎢                                                                             
⎣                    0                                         0              

                                                                              
                           0                       

In [6]:
from discretizer.algorithms import substitute_functions
from discretizer.algorithms import discretize

In [9]:
def magic(hamiltonian, space_dependent=None, discrete_coordinates=None,
          verbose=False, symbolic_output=False):

    tmp = substitute_functions(hamiltonian, discrete_coordinates, space_dependent)
    hamiltonian, discrete_coordinates = tmp

    if verbose:
        print('Discrete coordinates set to: ', sorted(discrete_coordinates))

    discrete_hamiltonian = discretize(hamiltonian, discrete_coordinates)

    if symbolic_output:
        return discrete_hamiltonian

    tb = make_kwant_functions(discrete_hamiltonian, discrete_coordinates, verbose)
    return tb

In [10]:
magic(hamiltonian, space_dependent=[A], verbose=True, symbolic_output=True)

Discrete coordinates set to:  ['x', 'y']
[C*Psi(x, y), M*Psi(x, y), -B*Psi(x, y)/(2*a_x**2) + B*Psi(-2*a_x + x, y)/(4*a_x**2) + B*Psi(2*a_x + x, y)/(4*a_x**2), -B*Psi(x, y)/(2*a_y**2) + B*Psi(x, -2*a_y + y)/(4*a_y**2) + B*Psi(x, 2*a_y + y)/(4*a_y**2), -D*Psi(x, y)/(2*a_x**2) + D*Psi(-2*a_x + x, y)/(4*a_x**2) + D*Psi(2*a_x + x, y)/(4*a_x**2), -D*Psi(x, y)/(2*a_y**2) + D*Psi(x, -2*a_y + y)/(4*a_y**2) + D*Psi(x, 2*a_y + y)/(4*a_y**2)]
[I*A(x, y)*Psi(-a_x + x, y)/(2*a_x) - I*A(x, y)*Psi(a_x + x, y)/(2*a_x), -A(x, y)*Psi(x, -a_y + y)/(2*a_y) + A(x, y)*Psi(x, a_y + y)/(2*a_y)]
[0]


UnboundLocalError: local variable 'index' referenced before assignment